In [1]:
import pandas as pd
#read data into data frame
date = pd.read_csv("date.csv")
patient = pd.read_csv("patient_info.csv")




In [2]:
# 1.1. The missing value from BMI can be replaced with the mean value of its regarding gender


#as a test, print all patients where BMI is NaN
print("Before:")
print(patient["BMI"][patient["BMI"].isna()])

#calculate mean by gender
female = patient.loc[patient["Sex"]=="Female"]
male = patient.loc[patient["Sex"]=="Male"]

meanBmiM = male["BMI"].mean();
meanBmiF = female["BMI"].mean();

#replace with mean
for x in patient.index:
    if pd.isna(patient.loc[x, "BMI"]):
        if (patient.loc[x, "Sex"]=="Female"):
            patient.loc[x, "BMI"] = meanBmiF
        else:
            patient.loc[x, "BMI"] = meanBmiM


#check if there is any NaN BMI data left
print("After:")
print(patient["BMI"][patient["BMI"].isna()])




Before:
93     NaN
263    NaN
369    NaN
803    NaN
828    NaN
958    NaN
1001   NaN
1052   NaN
1055   NaN
1060   NaN
1069   NaN
1141   NaN
1262   NaN
1356   NaN
1442   NaN
1793   NaN
1967   NaN
2132   NaN
2191   NaN
Name: BMI, dtype: float64
After:
Series([], Name: BMI, dtype: float64)


In [3]:
#1.2. Do the sanity check for all the columns in 2 files, and remove the sample which does not pass the sanity check

#PATIENT
print(patient.size)

#Sanity check for ID (must be 7 digits)
patient = patient[patient['Participant_ID'] >= 1000000]
print("After ID check :", patient.size)

#Sanity check for sex (must be "Male" or "Female")
patient = patient[patient["Sex"].isin(['Male', "Female"])]
print("After sex check: ", patient.size)

#Sanity check for age (can't be below 40)
patient = patient[patient['Age'] >= 40]
print("After age check: ", patient.size)

#Sanity check for Smoking Status (must be "Never", "Previous", "Current", "Prefer not to answer")
allowedSmokingStatus = ["Never", "Previous", "Current", "Prefer not to answer"]
patient = patient[patient["Smoking_status"].isin(allowedSmokingStatus)]
print("After Smoking check: ", patient.size)

#Sanity check for BMI (must be between 15 and 60)
patient = patient[patient['BMI'] >= 15]
patient = patient[patient['BMI'] <= 60]
print("After BMI check: ",patient.size)

#Sanity check for Disease (must be 1 or 2)
patient = patient[patient['Disease'].isin([1, 2])]
print("After disease check: ", patient.size)









13308
After ID check : 13308
After sex check:  13308
After age check:  13302
After Smoking check:  13284
After BMI check:  13272
After disease check:  13272


In [4]:
#DATE
print(date.size)
#Sanity check for ID (must be 7 digits)
date = date[date['Participant_ID'] >= 1000000]
print("After ID check :", date.size)

#Sanity check for VisitDate (must be after 1950-01-01 but before 2023-12-31)
date['VisitDate'] = pd.to_datetime(date['VisitDate'], format='%d/%m/%Y')

date = date[date["VisitDate"].isin(pd.date_range("1950-01-01", "2023-12-31"))]

print("After date check :", date.size)

5782
After ID check : 5782
After date check : 5610


In [5]:
# 1.3. Calculate the number of visits of each patient, if the VisitDate is missing, means they don't visit the hospital during that period

#count occurence of id in date, means get number of hospital visits for each id
Number_of_visit =  date.groupby("Participant_ID").count()
print(Number_of_visit.head(10))
print(Number_of_visit["VisitDate"].max())

                VisitDate
Participant_ID           
1003300                 2
1004025                 1
1005551                 2
1005594                 1
1005624                 2
1008540                 1
1008931                 2
1010012                 1
1010841                 2
1015236                 1
17


In [18]:
# 1.4. Merge two files with columns: Patient_ID, Sex, Age, BMI, Smoking_status, Number_of_visit, Disease

# make a new dataframe from patient and number of visits
# right means, if a patient does not have a hopital visit it is NaN
new_df= pd.merge(Number_of_visit, patient, on="Participant_ID", how="right")



# replace NaN with 0
new_df["VisitDate"].fillna(0, inplace = True)

# rename columns
new_df = new_df.rename(columns={'Participant_ID': 'Patient_ID', 'VisitDate': 'Number_of_visit'})


#reorder columns
new_df = new_df[["Patient_ID", "Sex", "Age", "BMI", "Smoking_status", "Number_of_visit", "Disease"]]
print(new_df.head(30))


    Patient_ID     Sex  Age      BMI Smoking_status  Number_of_visit  Disease
0      1003300    Male   62  33.7597       Previous              2.0        2
1      1004025    Male   60  28.1173       Previous              1.0        1
2      1005551    Male   69  26.4159       Previous              2.0        2
3      1005594    Male   64  24.7550       Previous              1.0        1
4      1005624    Male   64  35.0773       Previous              2.0        2
5      1008540  Female   64  21.6764       Previous              1.0        2
6      1008931    Male   62  26.9529       Previous              2.0        2
7      1010012    Male   66  33.8126          Never              1.0        2
8      1010841    Male   58  30.0826       Previous              2.0        1
9      1015236    Male   63  23.8804       Previous              1.0        2
10     1022720    Male   40  37.5521          Never              1.0        2
11     1030927    Male   69  24.9525       Previous             

In [7]:
# 2.1. Is the BMI different between the healthy and control group? make the box/violin chart

In [8]:
# 2.2. Is the proportion of Smoke status different between the healthy and control group?

In [9]:
# 2.3. Is the smoke status related to the number of Hospital visits regardless of the disease? (ignore the "Prefer not to answer")


In [10]:
# 2.4. Is the number of visits higher in the Disease group than in the Healthy group?


In [11]:
#2.5. Do Male has a higher current smoke percentage compared to Female?
